In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import Ridge, Lasso, ElasticNet, RidgeCV, LassoCV, ElasticNetCV
from joblib import dump, load

In [2]:
df = pd.read_csv('Advertising.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         200 non-null    float64
 1   radio      200 non-null    float64
 2   newspaper  200 non-null    float64
 3   sales      200 non-null    float64
dtypes: float64(4)
memory usage: 6.4 KB


In [4]:
X = df.drop('sales',axis=1)
y = df['sales']

In [5]:
poly_converter = PolynomialFeatures(degree=3,include_bias=False)
poly_feature = poly_converter.fit_transform(X)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(poly_feature, y, test_size=0.3, random_state=101)

In [7]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Ridge model

In [8]:
ridge_model = Ridge(alpha=10)

In [9]:
ridge_model.fit(X_train,y_train)

Ridge(alpha=10)

In [10]:
test_predictions = ridge_model.predict(X_test)

In [12]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = MSE**0.5

In [13]:
MAE

0.5774404204714183

In [14]:
RMSE

0.8946386461319685

In [15]:
training_predictions = ridge_model.predict(X_train)
MAE = mean_absolute_error(y_train,training_predictions)
MAE

0.5288348183025332

## Ridge with Cross-Validation

In [16]:
ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 10.0),scoring='neg_mean_absolute_error')

In [18]:
ridge_cv_model.fit(X_train,y_train)

RidgeCV(scoring='neg_mean_absolute_error')

In [19]:
ridge_cv_model.alpha_

0.1

In [20]:
test_predictions = ridge_cv_model.predict(X_test)

In [21]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = MSE**0.5

In [22]:
MAE

0.42737748843373746

In [24]:
RMSE

0.6180719926921404

In [30]:
training_predictions = ridge_cv_model.predict(X_train)
MAE = mean_absolute_error(y_train,training_predictions)
MAE

0.3094132105662787

In [26]:
ridge_cv_model.coef_

array([ 5.40769392,  0.5885865 ,  0.40390395, -6.18263924,  4.59607939,
       -1.18789654, -1.15200458,  0.57837796, -0.1261586 ,  2.5569777 ,
       -1.38900471,  0.86059434,  0.72219553, -0.26129256,  0.17870787,
        0.44353612, -0.21362436, -0.04622473, -0.06441449])

## Lasso with Cross-validation

In [27]:
lasso_cv_model = LassoCV(eps=0.1,n_alphas=100,cv=5)

In [28]:
lasso_cv_model.fit(X_train,y_train)

LassoCV(cv=5, eps=0.1)

In [29]:
lasso_cv_model.alpha_

0.4943070909225828

In [31]:
test_predictions = lasso_cv_model.predict(X_test)

In [32]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = MSE**0.5

In [33]:
MAE

0.6541723161252854

In [35]:
RMSE

1.130800102276253

In [36]:
training_predictions = lasso_cv_model.predict(X_train)
MAE = mean_absolute_error(y_train,training_predictions)
MAE

0.6912807140820695

In [37]:
lasso_cv_model.coef_

array([1.002651  , 0.        , 0.        , 0.        , 3.79745279,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

## Elastic Net

In [38]:
elastic_model = ElasticNetCV(l1_ratio=[.1, .5, .7,.9, .95, .99, 1],tol=0.01)

In [39]:
elastic_model.fit(X_train,y_train)

ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], tol=0.01)

In [40]:
elastic_model.l1_ratio_

1.0

In [41]:
test_predictions = elastic_model.predict(X_test)

In [42]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [43]:
MAE

0.5663262117569448

In [44]:
RMSE

0.7485546215633724

In [45]:
train_predictions = elastic_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.4307582990472369

In [46]:
elastic_model.coef_

array([ 3.78993643,  0.89232919,  0.28765395, -1.01843566,  2.15516144,
       -0.3567547 , -0.271502  ,  0.09741081,  0.        , -1.05563151,
        0.2362506 ,  0.07980911,  1.26170778,  0.01464706,  0.00462336,
       -0.39986069,  0.        ,  0.        , -0.05343757])